In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/visML_project/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!ls

BoT-SORT		  streetaware_sample_chase_1_sensor_1_left
bot_sort.ipynb		  streetaware_sample_chase_1_sensor_1_right
bytetrack		  streetaware_sample_chase_1_sensor_2_left
bytetrack_output_frames   streetaware_sample_chase_1_sensor_2_right
data_preprocessing.ipynb  streetaware_sample_chase_2_sensor_1_left
input			  streetaware_sample_chase_2_sensor_1_right
MOT17			  streetaware_sample_chase_2_sensor_2_left
MOT17-01-FRCNN.txt	  streetaware_sample_chase_2_sensor_2_right
output_frames		  yolov8l.pt
outputs			  yolov8m.pt
output_videos		  yolov8n.pt
right_people_track.json   yolov8x.pt
StreetAware


In [ ]:
input_paths = ['streetaware_sample_chase_1_sensor_1_left',
'streetaware_sample_chase_1_sensor_1_right',
 'streetaware_sample_chase_1_sensor_2_left',
 'streetaware_sample_chase_1_sensor_2_right',
 'streetaware_sample_chase_2_sensor_1_left',
 'streetaware_sample_chase_2_sensor_1_right',
 'streetaware_sample_chase_2_sensor_2_left',
 'streetaware_sample_chase_2_sensor_2_right'
 ]

In [ ]:
video_id = input_paths[0]
video = f'{video_id}/test/MOT17-01-FRCNN/img1'
video

'streetaware_sample_chase_1_sensor_1_left/test/MOT17-01-FRCNN/img1'

In [ ]:
# !pip install supervision ultralytics

In [ ]:
import supervision as sv
from ultralytics import YOLO
import cv2
from tqdm import tqdm
import argparse
import numpy as np
import json

In [ ]:
input_path = 'streetaware_sample_chase_1_sensor_1_right'
video_input_path = f'{input_path}/test/MOT17-01-FRCNN/img1'

In [ ]:
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# single frame example
model = YOLO('bytetrack/yolov8m.pt', task='predict')
tracker = sv.ByteTrack()

In [29]:
tracker.reset()
input_path = 'streetaware_sample_chase_1_sensor_1_left'
video_input_path = f'{input_path}/test/MOT17-01-FRCNN/img1'
frame_name = '000043.jpg'
frame = cv2.imread(f'{video_input_path}/{frame_name}')

results = model(frame, classes=[0])[0]

detections = sv.Detections.from_ultralytics(results)
detections = tracker.update_with_detections(detections)

labels = [f"ID: {tracker_id}, Conf: {str(round(conf,2))}" for tracker_id, conf in zip(detections.tracker_id, detections.confidence)]
print(labels)
if len(labels)==0:
  labels = ["#-1"]
print(len(labels))

annotated_frame = bounding_box_annotator.annotate(
        scene=frame.copy(), detections=detections)
annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels)


0: 480x640 7 persons, 2479.1ms
Speed: 14.9ms preprocess, 2479.1ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
['ID: 1, Conf: 0.91', 'ID: 2, Conf: 0.88', 'ID: 3, Conf: 0.84', 'ID: 4, Conf: 0.7', 'ID: 5, Conf: 0.69', 'ID: 6, Conf: 0.63', 'ID: 7, Conf: 0.56']
7


In [24]:
int(detections.box_area[0])

56931

In [25]:
from google.colab.patches import cv2_imshow

In [26]:
# cv2_imshow(frame)

In [27]:
# labels

In [30]:
cv2_imshow(annotated_frame)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def box_overlap_count(boxes):
    def overlap(box1, box2):
        # Check if box1 overlaps with box2
        x1_min, y1_min, x1_max, y1_max = box1
        x2_min, y2_min, x2_max, y2_max = box2
        return not (x1_max < x2_min or x2_max < x1_min or y1_max < y2_min or y2_max < y1_min)

    count = 0
    for i in range(len(boxes)):
        for j in range(i + 1, len(boxes)):
            if overlap(boxes[i], boxes[j]):
                count += 1
    return count

In [ ]:
for loc_path in ['chase_1','chase_2']:
  for sensor_path in ['sensor_1','sensor_2']:
    for side in ['right','left']:
      folder_path = f'bytetrack_output_frames/{loc_path}/{sensor_path}_{side}'
      if not os.path.exists(folder_path):
        os.makedirs(folder_path)
      print(f'bytetrack_output_frames/{loc_path}/{sensor_path}_{side}')

bytetrack_output_frames/chase_1/sensor_1_right
bytetrack_output_frames/chase_1/sensor_1_left
bytetrack_output_frames/chase_1/sensor_2_right
bytetrack_output_frames/chase_1/sensor_2_left
bytetrack_output_frames/chase_2/sensor_1_right
bytetrack_output_frames/chase_2/sensor_1_left
bytetrack_output_frames/chase_2/sensor_2_right
bytetrack_output_frames/chase_2/sensor_2_left


In [ ]:
def process_frames(location_path = None, sensor_path = None, side_path = None):
  tracker.reset()
  video_input_path = f'streetaware_sample_{location_path}_{sensor_path}_{side_path}/test/MOT17-01-FRCNN/img1'
  output_path = f'bytetrack_output_frames/{loc_path}/{sensor_path}_{side_path}'

  print(f'Processing: {video_input_path} ')
  print(f'Output dir: {output_path}')
  frame_i = 1

  frame_outputs = []
  sink = sv.ImageSink(target_dir_path=f'{output_path}',
                    overwrite=True)

  frames_ = os.listdir(video_input_path)
  frames_.sort()

  for frame_name in frames_:
    print(f'{video_input_path}/{frame_name}')
    frame = cv2.imread(f'{video_input_path}/{frame_name}')

    results = model(frame, classes=[0])[0]
    print(f'Frame {frame_i}, {frame_name}')
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    n_overlaps = box_overlap_count(detections.xyxy)
    # print(detections)
    print(f'N detections: {len(detections.class_id)}, {n_overlaps}')
    for obj_i in range(0,len(detections.class_id)):
      if len(detections.tracker_id)>0:
        id = detections.tracker_id[obj_i]
      else:
        id = -1
      bounding_box = detections.xyxy[obj_i].flatten()
      score = detections.confidence[obj_i]
      bbox_area = detections.box_area[obj_i]

      struct_results = {
        'video': input_path,
        'frame': int(frame_i),
        'people_id': int(id),
        'score': str(round(score,4)),
        'bbox': [ str(round(elem,2)) for elem in list(bounding_box) ],
        'bbox_area': int(bbox_area),
        'n_overlapping_boxes': n_overlaps
        }
      frame_outputs.append(struct_results)

      # struct_results = np.append(struct_results,frame_i)
      # struct_results = np.append(struct_results,id)
      # struct_results = np.append(struct_results,bounding_box)
      # struct_results = np.append(struct_results, score)

      # break
      # print(struct_results)
    labels = [f"ID: {tracker_id}, Conf: {str(round(conf,2))}" for tracker_id, conf in zip(detections.tracker_id, detections.confidence)]
    print(labels)
    if len(labels)==0:
      labels=[f"#-{tracker_id}" for tracker_id in detections.class_id]
    print(f'N labels: {len(labels)}')

    # annotated_frame = bounding_box_annotator.annotate(
    #     scene=frame.copy(), detections=detections)
    # annotated_frame = label_annotator.annotate(
    #     scene=annotated_frame, detections=detections, labels=labels)

    # cv2.imwrite(f'annotated_img1/annotated_{frame_name}', annotated_frame)
    # sink.save_image(annotated_frame, f'{frame_name}')

    frame_i+=1
    # print(frame_outputs)
    # break
  with open(f'outputs/bytetrack/{location_path}_{sensor_path}_{side_path}_outcome.json', 'w') as f:
    json.dump(frame_outputs, f)


In [ ]:
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# single frame example
model = YOLO('yolov8l.pt', task='predict')
tracker = sv.ByteTrack()

In [ ]:
# process_frames(location_path='chase_1',sensor_path='sensor_1',side_path='right')

In [ ]:
for loc_path in ['chase_1']:
  for sensor_path in ['sensor_1','sensor_2']:
    for side in ['right','left']:
      folder_path = f'bytetrack_output_frames/{loc_path}/{sensor_path}_{side}'
      # if folder_path in ['bytetrack_output_frames/chase_1/sensor_1_right',
      #                    'bytetrack_output_frames/chase_1/sensor_1_left',
      #                    'bytetrack_output_frames/chase_1/sensor_2_left',
      #                    'bytetrack_output_frames/chase_1/sensor_2_right',
      #                    ]:
      #   print('passing', folder_path)
      #   continue
      if not os.path.exists(folder_path):
        os.makedirs(folder_path)
      process_frames(loc_path,sensor_path,side)

Streaming output truncated to the last 5000 lines.
streetaware_sample_chase_1_sensor_2_left/test/MOT17-01-FRCNN/img1/000276.jpg

0: 480x640 4 persons, 2370.5ms
Speed: 4.9ms preprocess, 2370.5ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)
Frame 276, 000276.jpg
N detections: 4, 1
['ID: 28, Conf: 0.77', 'ID: 20, Conf: 0.71', 'ID: 13, Conf: 0.7', 'ID: 4, Conf: 0.51']
N labels: 4
streetaware_sample_chase_1_sensor_2_left/test/MOT17-01-FRCNN/img1/000277.jpg

0: 480x640 4 persons, 3525.8ms
Speed: 5.4ms preprocess, 3525.8ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)
Frame 277, 000277.jpg
N detections: 4, 1
['ID: 28, Conf: 0.77', 'ID: 13, Conf: 0.72', 'ID: 20, Conf: 0.65', 'ID: 4, Conf: 0.56']
N labels: 4
streetaware_sample_chase_1_sensor_2_left/test/MOT17-01-FRCNN/img1/000278.jpg

0: 480x640 3 persons, 2400.2ms
Speed: 5.0ms preprocess, 2400.2ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)
Frame 278, 000278.jpg
N detections: 3, 0
['I

In [ ]:
# frame_outputs

In [ ]:
# np.savetxt('bytetrack_outcome.txt',np.around(final_output_arr,2))


In [ ]:
# detections.class_id, detections.xyxy, detections.confidence

In [ ]:
# struct_results

In [ ]:
# frame, id, boundind box, score